In [15]:
import requests
import base64
import json
import time
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv 
# --- 1. Configuration ---
load_dotenv()  # <-- Load variables from your .env file

# Read the API key from the environment; exit if not found
api_key = os.getenv("RUNPOD_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please create a .env file and add RUNPOD_API_KEY=your_key")


endpoint_id = "bsze1gjo0ob5lk"

# --- 2. Define the API URLs ---
run_url = f"https://api.runpod.ai/v2/{endpoint_id}/run"
status_url_template = f"https://api.runpod.ai/v2/{endpoint_id}/status/"

# --- 3. Define the FLUX Workflow Payload ---
flux_workflow = {
    "input": {
        "workflow": {
            "3": {"inputs": {"seed": 1337, "steps": 28, "cfg": 1.0, "sampler_name": "dpmpp_2m", "scheduler": "simple", "denoise": 1.0, "model": ["4", 0], "positive": ["6", 0], "negative": ["7", 0], "latent_image": ["5", 0]}, "class_type": "KSampler"},
            "4": {"inputs": {"ckpt_name": "flux1-dev-fp8.safetensors"}, "class_type": "CheckpointLoaderSimple"},
            "5": {"inputs": {"width": 1024, "height": 1024, "batch_size": 1}, "class_type": "EmptyLatentImage"},
            "6": {"inputs": {"text": "Vibrant and fun digital art illustration of the Golang mascot, a cute cartoon gopher. He is cheerfully wearing a white Hooters tank top and enthusiastically holding up a frosty, overflowing pitcher of beer. The style is a clean, smooth digital painting with bright colors and sharp focus, trending on ArtStation. The background is simple and uncluttered to keep the focus on the character", "clip": ["4", 1]}, "class_type": "CLIPTextEncode"},
            "7": {"inputs": {"text": "photograph, realistic, 3d render, blurry, ugly, deformed, disfigured, watermark, text, signature, bad anatomy, extra limbs, poorly drawn, dark, scary", "clip": ["4", 1]}, "class_type": "CLIPTextEncode"},
            "8": {"inputs": {"samples": ["3", 0], "vae": ["4", 2]}, "class_type": "VAEDecode"},
            "9": {"inputs": {"filename_prefix": "ComfyUI_FP8_Checkpoint", "images": ["8", 0]}, "class_type": "SaveImage"}
        }
    }
}

# --- 4. Send the Initial API Request to Start the Job ---
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

print("Sending request to start the job...")
response = requests.post(run_url, headers=headers, json=flux_workflow)

if response.status_code == 200:
    response_data = response.json()
    job_id = response_data.get('id')
    
    if not job_id:
        print("Error: API response did not include a job ID.")
        print("Full response:", json.dumps(response_data, indent=2))
    else:
        print(f"Job successfully started with ID: {job_id}")

        # --- 5. Poll for the Job Status ---
        status_url = status_url_template + job_id
        while True:
            print("Polling for job status...")
            status_response = requests.get(status_url, headers=headers)
            status_data = status_response.json()
            job_status = status_data.get('status')

            if job_status == 'COMPLETED':
                print("Job completed successfully!")
                
                # --- 6. Process the Final Output ---
                try:
                    output_image_data = status_data['output']['images'][0]

                    image_base64 = output_image_data['data'] 
                    filename = output_image_data['filename']

                    image_bytes = base64.b64decode(image_base64)
                    image = Image.open(BytesIO(image_bytes))
                    
                    image.save(filename)
                    print(f"\n Image successfully generated and saved as '{filename}'\n")

                except (KeyError, IndexError, TypeError) as e:
                    print(f"\n Error: Could not parse image data from the final response: {e}\n")
                    
                    debug_data = json.loads(json.dumps(status_data))
                    try:
                        # Correctly look for the 'data' key to redact it
                        for img in debug_data['output']['images']:
                            if 'data' in img:
                                original_length = len(img['data'])
                                img['data'] = f"<base64 data redacted, original length: {original_length}>"
                    except Exception:
                        pass 
                        
                    print("Full response (redacted):", json.dumps(debug_data, indent=2))
                
                break

            elif job_status in ['IN_QUEUE', 'IN_PROGRESS']:
                time.sleep(5)
            else:
                print(f"Job execution failed or was cancelled.")
                print("Final Status:", job_status)
                if 'output' in status_data:
                    print("Output/Error:", status_data['output'])
                else:
                    print("Full response:", json.dumps(status_data, indent=2))
                break 

else:
    print(f"Error: Initial API request failed with status code {response.status_code}")
    print("Response:", response.text)

Sending request to start the job...
Job successfully started with ID: 460a8628-4f6d-4f04-a42f-c29b9d17c283-u2
Polling for job status...
Polling for job status...
Polling for job status...
Polling for job status...
Polling for job status...
Job completed successfully!

 Image successfully generated and saved as 'ComfyUI_FP8_Checkpoint_00006_.png'

